#### 설정값 선언


In [1]:
!pip install ratsnlp -q

     |████████████████████████████████| 42 kB 348 kB/s 
     |████████████████████████████████| 2.8 MB 39.8 MB/s 
     |████████████████████████████████| 582 kB 59.9 MB/s 
     |████████████████████████████████| 57 kB 2.6 MB/s 
     |████████████████████████████████| 529 kB 50.5 MB/s 
     |████████████████████████████████| 3.3 MB 21.1 MB/s 
     |████████████████████████████████| 880 kB 58.8 MB/s 
     |████████████████████████████████| 163 kB 63.0 MB/s 
     |████████████████████████████████| 96 kB 1.4 MB/s 


In [2]:
from ratsnlp.nlpbook.classification import ClassificationTrainArguments

In [3]:
args = ClassificationTrainArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_corpus_name="nsmc",
    downstream_corpus_root_dir="/content/Korpora",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-doccls",
    learning_rate=5e-5,
    batch_size=32
)
# "nsmc" ; 네이버 영화 리뷰 말뭉차
# 파인튜닝을 하기 위해 다운스트림 데이터를 미리 다운받아야함
# nsmc를 코랩 환결 로컬의 /content/Korpora 디렉터리에 저장

#### 데이터 내려받기

In [4]:
from Korpora import Korpora
Korpora.fetch(
    corpus_name=args.downstream_corpus_name,
    root_dir=args.downstream_corpus_root_dir,#
    force_download=True,
)
# downstream_corpus_name에 해당하는 말뭉치를 내려받아 downstream_corpus_root_dir 아래에 저장

[nsmc] download ratings_train.txt: 14.6MB [00:00, 251MB/s]
[nsmc] download ratings_test.txt: 4.90MB [00:00, 140MB/s]


#### 프리트레인을 마친 모델 준비하기
- 프리트레인 : 업스트림 태스크를 학습하는 과정

In [5]:
from transformers import BertConfig, BertForSequenceClassification
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=2,
)
model = BertForSequenceClassification.from_pretrained(
    args.pretrained_model_name,
    config=pretrained_model_config,
)
# kcbert-base가 로컬 저장소에 없으면 자동으로 내려받고, 있으면 캐시 디렉터리에서 읽어옵니다.

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

#### 토크나이저 준비하기
- 자연어 처리 모델의 입력은 대개 토큰
- 토큰은 문장보다 작은 단위
- 토큰 분리 기준 ( 상황마다 다름 ) : 띄어쓰기, 형태소

- 토큰화 : 문장을 토큰 시퀀스로 분석하는 과정
- 토크나이저 : 토큰화를 수행하는 프로그램

In [6]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)
# kcbert_base 모델이 사용하는 토크나이저를 준비하는 코드
# 토크나이저 관련 파일이 로컬 저장소에 없으면 자동으로 내려받고, 있으면 캐시에서 읽어옵니다.

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

#### 데이터 로더 준비하기 
- 파이토치에는 데이터 로더가 포함되어 있다.
- 데이터로더 : 데이터를 배치 단위로 밀어 넣어주는 역할을 한다.
   - 1) 전체 데이터 가운데 일부 인스턴스를 뽑아 배치를 구성
   - 2) 데이터셋은 데이터 로더의 구성 요소 가운데 하나
   - 3) 배치는 모양이 고정적이어야 할 때가 많다. ( 동일한 배치에 있는 문장의 토큰 개수가 같아야 한다. )
   - 4) 컬레이트 : 배치의 모양등을 정비해 모델의 최종 입력으로 만들어주는 과정

In [7]:
from ratsnlp import nlpbook
from torch.utils.data import DataLoader, RandomSampler
from ratsnlp.nlpbook.classification import NsmcCorpus, ClassificationDataset
corpus = NsmcCorpus()
train_dataset = ClassificationDataset(
    args = args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)

In [8]:
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

#### 태스크 정의하기
- 모델학습을 위해 파이토치 라이트닝 ( 딥러닝 모델 학습 시 반복적인 내용을 대신 수행 )
- 최적화 : 특정 조건에서 어떤 값이 최대나 최소가 되도록 하는 과정 ( 옵티마이저, 러닝 레이트 스케줄러 )

- 모델 학습은 배치 단위
- 스텝 : 배치를 모델에 입력한 뒤 모델 출력을 정답과 비교해 차이를 계산합니다. 이후 차이를 최소화하는 방향으로 모델으로 업데이트, 이 과정을 순환

#### 모델 학습하기
- 파이토치 라이트닝에서 제공하는 객체로 실제 학습을 수행
- GPU 등 하드웨어 설정, 학습 기록 로깅, 체크포인트 저장 등 복잡한 설정들을 알아서 해줍니다.

In [10]:
from ratsnlp.nlpbook.classification import ClassificationTask

task = ClassificationTask(model, args)
trainer = nlpbook.get_trainer(args)
trainer.fit(
    task,
    train_dataloaders=train_dataloader
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:608: UserWarning: Checkpoint directory /gdrive/My Drive/nlpbook/checkpoint-doccls exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                          | Params
--------------------------------------------------------
0 | model | BertForSequenceClassification | 108 M 
--------------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M   

Training: 0it [00:00, ?it/s]